In [ ]:
import pandas as pd
import os

# Import GenoTools library for Wrayner tool
from genotools.utils import shell_do, merge_genos
from genotools.imputation import *

In [ ]:
# Release of interest
rel = 10

# Create folders to mount
! mkdir releases release ref_files working

# Read Access only
! gcsfuse --dir-mode 555 --file-mode 444 --implicit-dirs gp2tier2_vwb releases
! gcsfuse --dir-mode 555 --file-mode 444 --implicit-dirs gp2_release{rel} release
! gcsfuse --dir-mode 555 --file-mode 444 --implicit-dirs gtserver-eu-west4-gp2-release-terra ref_files

# Read/Write Access
! gcsfuse --dir-mode 777 --file-mode 777 --implicit-dirs gp2_working_eu working

In [ ]:
wd = '/YOUR/WORKING/DIR'

### Create Files with Sex Info

In [ ]:
# Keep subset of IDs based on sex
label = 'AJ'
user = 'nicole'
sex = 'male'

! mkdir {wd}/working/{user}/x_chrom_new/imputation/{label}

master = pd.read_csv(f'{wd}/releases/release{rel}/clinical_data/master_key_release{rel}_final_vwb.csv')
ancestry = master[(master.nba_label == label) & (master.biological_sex_for_qc == sex.title())]
ancestry['FID'] = 0
ancestry[['FID', 'GP2ID']].to_csv(f'{wd}/working/{user}/x_chrom_new/imputation/{label}/{label}_release{rel}_{sex}.samples', sep = '\t', index = False, header = None)

# Check if any samples were pruned
display(ancestry.nba_prune_reason.value_counts())

In [ ]:
# For EUR: Split key into 3 parts due to sample limits on impuation
label = 'EUR'
sex = 'male'

# Shuffle the DataFrame within each phenotype group
ancestry = master[(master.nba_label == label) & (master.biological_sex_for_qc == sex.title())]
df = ancestry.groupby('GP2_phenotype_for_qc', group_keys=False).apply(lambda x: x.sample(frac=1, random_state=42)).reset_index(drop=True)

# Split each phenotype group into three equal parts
grouped = df.groupby('GP2_phenotype_for_qc')
df1 = pd.concat([grp.iloc[:len(grp) // 3] for _, grp in grouped])
df2 = pd.concat([grp.iloc[len(grp) // 3:2 * len(grp) // 3] for _, grp in grouped])
df3 = pd.concat([grp.iloc[2 * len(grp) // 3:] for _, grp in grouped])

df1['FID'] = 0
df2['FID'] = 0
df3['FID'] = 0

# Check phenotype counts in each part
print("Part 1 counts:\n", len(df1), df1['GP2_phenotype_for_qc'].value_counts(), '\n')
print("Part 2 counts:\n", len(df2), df2['GP2_phenotype_for_qc'].value_counts(), '\n')
print("Part 3 counts:\n", len(df3), df3['GP2_phenotype_for_qc'].value_counts())

df1[['FID', 'GP2ID']].to_csv(f'{wd}/working/{user}/x_chrom_new/imputation/EUR_part1/EUR_part1_release{rel}_{sex}.samples', sep = '\t', index = False, header = None)
df2[['FID', 'GP2ID']].to_csv(f'{wd}/working/{user}/x_chrom_new/imputation/EUR_part2/EUR_part2_release{rel}_{sex}.samples', sep = '\t', index = False, header = None)
df3[['FID', 'GP2ID']].to_csv(f'{wd}/working/{user}/x_chrom_new/imputation/EUR_part3/EUR_part3_release{rel}_{sex}.samples', sep = '\t', index = False, header = None)

### Explore Release-Wide GenoTools Outputs

In [ ]:
# EUR run can begin here because ID subsets were pre-made -- ADD CODE FOR SPLITTING EUR INTO 3 CHUNKS
user = 'nicole'
sex = 'female'
group = f'r{rel}_final'
label = 'EUR_part1' # Can also choose: EUR_part2, EUR_part3

In [ ]:
keep_file = f'{wd}/working/{user}/x_chrom_new/imputation/{label}/{label}_release{rel}_{sex}.samples'

# Check that XY and X Regions Exist
test = pd.read_csv(f'{wd}/release/{group}/GP2_{group}_post_genotools_{label}.pvar', sep = '\s+', low_memory = False)
df2 = pd.DataFrame(test['#CHROM'].value_counts()).reset_index()
chrom_list = ['X', 'Y', 'XY', 'MT']
df2 = df2[df2['#CHROM'].isin(chrom_list)]
display(df2)

In [ ]:
# Split into sex of interest
input_file = f'{wd}/release/{group}/GP2_{group}_post_genotools_{label}'
output_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}'

# Change directory to /dev/shm for more stoage + easy cleanup for tmp files
os.chdir('/dev/shm')

! mkdir imputation imputation/{group}
! plink2 --pfile {input_file} --keep {keep_file} --make-pgen --out {output_file}

In [ ]:
# Double check that output is one sex
check_bim = pd.read_csv(f'{output_file}.psam', sep = '\t')
display(check_bim.SEX.value_counts())

### Region-Specific QC for PAR and Non-PAR

In [ ]:
region = 'PAR'

input_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}'
output_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}'

In [ ]:
# Subset by region
if region == 'non_PAR':
    ! plink2 --pfile {input_file} --chr X --set-invalid-haploid-missing --make-pgen --out {output_file}
elif region == 'PAR':
    ! plink2 --pfile {input_file} --chr XY --make-pgen --out {output_file}

In [ ]:
if region == 'PAR':
    # Load the .bim file
    pvar_file = f'{output_file}.pvar'
    pvar = pd.read_csv(pvar_file, sep="\t", low_memory = False)

    # Replace chromosome names
    pvar['#CHROM'] = pvar['#CHROM'].replace({'XY': 'X'})
    display(pvar['#CHROM'].value_counts())

    # Save the updated .bim file
    pvar.to_csv(f'{output_file}.pvar', sep="\t", index=False)

    # Split-PAR and remove any remaining X
    input_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}'
    output_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}_only'

    ! plink2 --pfile {input_file} --split-par b38 --make-pgen --out {output_file}

    # Only keep PAR1 and PAR2
    input_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}_only'
    output_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}_only_fixed'

    ! plink2 --pfile {input_file} --chr PAR1,PAR2 --make-pgen --out {output_file}

    # Load the .bim file again
    pvar_file = f'{output_file}.pvar'
    pvar = pd.read_csv(pvar_file, sep="\t", low_memory = False)

    # Replace chromosome names to avoid removal during imputation
    display(pvar['#CHROM'].value_counts())
    pvar['#CHROM'] = pvar['#CHROM'].replace({'PAR1': 'X', 'PAR2': 'X'})
    display(pvar['#CHROM'].value_counts())

    # Save the updated .bim file
    pvar.to_csv(f'{output_file}.pvar', sep="\t", index=False)

In [ ]:
# Finalize PLINK files per region
if region == 'PAR':
    input_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}_only_fixed'
elif region == 'non_PAR':
    input_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}'
output_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_{region}_final'

! plink2 --pfile {input_file} --output-chr 26 --make-bed --out {output_file}

### Save files to permanent storage in Google Buckets

In [ ]:
if sex == 'female':
    par_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_PAR_final'
    non_par_file = f'/dev/shm/imputation/{group}/GP2_{group}_post_genotools_{label}_{sex}_non_PAR_final'

    # Check if they exist
    if os.path.exists(f'{par_file}.bed') and os.path.exists(f'{non_par_file}.bed'):
        print("Both PAR and non-PAR files exist for females! Ready to merge.\n")
    elif os.path.exists(f'{par_file}.bed'):
        print("Only PAR file exists for females. Please prepare female non-PAR file before moving forward.\n")
    elif os.path.exists(f'{non_par_file}.bed'):
        print("Only Non-PAR file exists for females. Please prepare female PAR file before moving forward.\n")
    else:
        print("Neither file exists.")

    # Merge PAR and non-PAR regions for females
    output_file = f'GP2_{group}_post_genotools_{label}_{sex}_final'
    ! plink --bfile {non_par_file} --bmerge {par_file} --make-bed --out {output_file}

In [ ]:
# Copy over final output to Bucket storage
! mkdir {wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}

if sex == 'male':
    ! mkdir {wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}/{region}
    ! scp {output_file}.* {wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}/{region}
elif sex == 'female':
    if not os.path.exists(f'{output_file}.bed'):
        print("Please make sure that both PAR and Non-PAR files are created and merged for females.")
    else:
        ! scp {output_file}.* {wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}


### Submit to Wrayner Tool for VCF Creation

In [ ]:
ancestry_list = [label]
# ancestry_list = ['EUR_part1', 'EUR_part2', 'EUR_part3'] # if the above sections were completed for all 3 parts of EUR

impute_genos_list = []
impute_labels_list = []
for label in ancestry_list:
    if sex == 'male':
        path_to_file = f'{wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}/{region}/GP2_{group}_post_genotools_{label}_{sex}_{region}_final'
    elif sex == 'female':
        path_to_file = f'{wd}/working/{user}/x_chrom_new/imputation/{label}/{sex}/GP2_{group}_post_genotools_{label}_{sex}_final'
    impute_genos_list.append(path_to_file)
    impute_labels_list.append(label)
print(impute_genos_list)
print(impute_labels_list)

In [ ]:
# TOPMed Preparation
ref_dir = f'{wd}/ref_files/clinical'

topmed_ref_panel= f'{ref_dir}/PASS.Variants.TOPMed_freeze5_hg38_dbSNP.tab.gz'
check_bim_pl = f'{ref_dir}/HRC-1000G-check-bim.pl'

for geno, label in zip(impute_genos_list, impute_labels_list):   
    if sex == 'male':
        imputed_out_dir = f'{wd}/working/nicole/x_chrom_new/imputation/imputation_out_dir/{sex}/{region}/'
        imputation_prep = f'{wd}/working/nicole/x_chrom_new/imputation/imputation_prep/{sex}/{region}/'
    elif sex == 'female':
        imputed_out_dir = f'{wd}/working/nicole/x_chrom_new/imputation/imputation_out_dir/{sex}/'
        imputation_prep = f'{wd}/working/nicole/x_chrom_new/imputation/imputation_prep/{sex}/'

    label_outdir = f'{imputed_out_dir}/{label}'
    
    impute_prep_outdir = f'{imputation_prep}/{label}'
    impute_prep_geno = f'{impute_prep_outdir}/{label}'

    os.makedirs(f'{label_outdir}', exist_ok=True)
    os.makedirs(f'{impute_prep_outdir}', exist_ok=True)
    
    impute_data = impute_data_prep(geno, impute_prep_geno, topmed_ref_panel, check_bim_pl)

In [ ]:
# Check that files were made
for label in ancestry_list:
    ! ls -lh {imputation_prep}/{label}/{label}_pre_impute_chr{23}.vcf.gz

### VCF Prep

In [ ]:
# Write a chr map file to rename Chr23 to X
with open(f"{wd}/chr_map.txt", "w") as f:
    f.write("chr23 chrX\n")

In [ ]:
# Adjust Chromosome 23
for label in ancestry_list:
      out_folder = f'{imputation_prep}/{label}/'
      ! tabix -p vcf {out_folder}/{label}_pre_impute_chr23.vcf.gz -f

      # Rename
      ! bcftools annotate --rename-chrs {wd}/chr_map.txt -Oz -o {out_folder}/{label}_23_output.vcf.gz {out_folder}/{label}_pre_impute_chr23.vcf.gz
      
      # Normalize
      ! bcftools norm --check-ref s --fasta-ref {wd}/ref_files/clinical/hg38.fa {out_folder}/{label}_23_output.vcf.gz -Oz -o {out_folder}/{label}_pre_impute_chr23_fixed.vcf.gz

In [ ]:
# Double check that VCF files exist to submit to imputation
for label in ancestry_list:
    ! ls -lh {imputation_prep}/{label}/{label}_pre_impute_chr23_fixed.vcf.gz

### Submit VCF to Imputation

In [ ]:
# Double check commands to send to imputation

for label in ancestry_list:
    if sex == 'female':
        region = 'full'
    impute_cmd = f'imputationbot impute --files {imputation_prep}/{label}/{label}_pre_impute_chr23_fixed.vcf.gz --refpanel topmed-r3 --build hg38 --r2Filter 0.3 --population all --name x_chrom_{label}_{sex}_{region}'
    print(impute_cmd)

In [ ]:
# Submit job

for label in ancestry_list:
    ! {impute_cmd}